In [1]:
# %matplotlib qt

import matplotlib.pyplot as plt
from poisson_fem import PoissonFEM
import ROM
import GenerativeSurrogate as gs
import Data as dta
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc
import torch
from torch import optim

In [2]:
# Some fixed parameters
lin_dim_rom = 4                      # Linear number of rom elements
a = np.array([1, 2, 3])              # Boundary condition function coefficients
dim_z = 100                            # Latent space dimension
dtype = torch.float                  # Tensor data type

In [3]:
# Define mesh and boundary conditions
mesh = PoissonFEM.RectangularMesh(np.ones(lin_dim_rom)/lin_dim_rom)
# mesh.plot()

def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps

def essBoundaryFun(x):
    return 0.0
mesh.setEssentialBoundary(origin, essBoundaryFun)

def domainBoundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.setNaturalBoundary(domainBoundary)

def flux(x):
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [4]:
#Spepify right hand side and stiffness matrix
#Define boundary flux field
rhs = PoissonFEM.RightHandSide(mesh)
rhs.setNaturalRHS(mesh, flux)
funSpace = PoissonFEM.FunctionSpace(mesh)
K = PoissonFEM.StiffnessMatrix(mesh, funSpace)
rhs.setRhsStencil(mesh, K)

In [5]:
# Set up solver
ksp = PETSc.KSP().create()
ksp.setType('preonly')
precond = ksp.getPC()
precond.setType('cholesky')
ksp.setFromOptions() #???

In [6]:
# define rom
rom = ROM.ROM(mesh, K, rhs, ksp)

In [7]:
trainingData = dta.StokesData(range(512))
trainingData.readData(['IMG'])
# trainingData.plotMicrostruct(1)

In [8]:
model = gs.GenerativeSurrogate(rom, trainingData, dim_z)

In [9]:
steps = int(150)
for s in range(steps):
    print('step = ', s)
    batchSamples = torch.LongTensor(model.batchSizeN).random_(0, trainingData.nSamples)
    model.pfStep(batchSamples)

step =  0
loss =  tensor(5179394., grad_fn=<SubBackward0>)
step =  1
loss =  tensor(5141770.5000, grad_fn=<SubBackward0>)
step =  2
loss =  tensor(5105520., grad_fn=<SubBackward0>)
step =  3
loss =  tensor(5043710., grad_fn=<SubBackward0>)
step =  4
loss =  tensor(5039710., grad_fn=<SubBackward0>)
step =  5
loss =  tensor(4973045., grad_fn=<SubBackward0>)
step =  6
loss =  tensor(4962454.5000, grad_fn=<SubBackward0>)
step =  7
loss =  tensor(4916611.5000, grad_fn=<SubBackward0>)
step =  8
loss =  tensor(4883929., grad_fn=<SubBackward0>)
step =  9
loss =  tensor(4866855.5000, grad_fn=<SubBackward0>)
step =  10
loss =  tensor(4828365.5000, grad_fn=<SubBackward0>)
step =  11
loss =  tensor(4814273.5000, grad_fn=<SubBackward0>)
step =  12
loss =  tensor(4780096., grad_fn=<SubBackward0>)
step =  13
loss =  tensor(4746748.5000, grad_fn=<SubBackward0>)
step =  14
loss =  tensor(4745533., grad_fn=<SubBackward0>)
step =  15
loss =  tensor(4708643., grad_fn=<SubBackward0>)
step =  16
loss =  ten

loss =  tensor(4481261.5000, grad_fn=<SubBackward0>)
step =  134
loss =  tensor(4571911.5000, grad_fn=<SubBackward0>)
step =  135
loss =  tensor(4525618.5000, grad_fn=<SubBackward0>)
step =  136
loss =  tensor(4528813., grad_fn=<SubBackward0>)
step =  137
loss =  tensor(4492880., grad_fn=<SubBackward0>)
step =  138
loss =  tensor(4503380., grad_fn=<SubBackward0>)
step =  139
loss =  tensor(4596701., grad_fn=<SubBackward0>)
step =  140
loss =  tensor(4520092., grad_fn=<SubBackward0>)
step =  141
loss =  tensor(4529584., grad_fn=<SubBackward0>)
step =  142
loss =  tensor(4512538., grad_fn=<SubBackward0>)
step =  143
loss =  tensor(4541815.5000, grad_fn=<SubBackward0>)
step =  144
loss =  tensor(4498305.5000, grad_fn=<SubBackward0>)
step =  145
loss =  tensor(4504824., grad_fn=<SubBackward0>)
step =  146
loss =  tensor(4520464., grad_fn=<SubBackward0>)
step =  147
loss =  tensor(4550431.5000, grad_fn=<SubBackward0>)
step =  148
loss =  tensor(4519565., grad_fn=<SubBackward0>)
step =  149


In [10]:
model.plotInputReconstruction()

In [11]:
f = plt.gcf()
f.suptitle('Untrained, N = 2048', fontsize=32, y=.7)

Text(0.5, 0.7, 'Untrained, N = 2048')

In [12]:
trainingData.imgResolution**2*32*np.log(2)

1453634.9960056504

In [13]:
trainingData

In [14]:
trainingData.microstructImg.nelement()

33554432

In [15]:
X = torch.zeros(int(1e4), int(1e5))

In [16]:
X.shape

torch.Size([10000, 100000])

In [17]:
X.unsqueeze(2)
X = X.expand(int(1e4), int(1e5))

In [18]:
X.shape

torch.Size([10000, 100000])